# Лабораторная работа 4
Регрессия и классификация в sklearn. Метрики регрессии классификации

Данная лабораторная выполняет в командах по 3-4 человека. Датасеты у команд не должны пересекаться.

## Регрессия

Ссылки на теорию:

- https://habr.com/ru/company/ods/blog/323890/
- https://habr.com/ru/post/278513/
- https://habr.com/ru/post/279117/

0. Загрузить датасет, подходящий для задачи регрессии. Набор данных должен содержать не только числовые, но и категориальные/бинарные признаки. Опишите данные, что обозначает каждый столбец, какого типа (категориальный, вещественный, целочисленный, бинарный) данный столбец, при необходимости для категориальных в том числе бинарных признаков привести расшифровку их значений.


1. Удалить категориальные / бинарные признаки, оставив только вещественные.

2. Визуализировать heatmap-матрицу кореллированности признаков.

3. Визуализировать взаимосвязь между целевой переменной и каждым признаком (с помощью sns.pairplot или pandas.plotting.scatter_matrix или как-то ещё). Лучше всего, если будете строить диаграмму попарного распределения не между ВСЕМИ парами признаков, а только между признаками и целевой переменной.

4. Разделить набор данных на обучающую и тестовую выборки

##### 5. Линейная регрессия

5.1 Обучить модель линейной регрессии (LinearRegression)

5.2 Вывести метрики MAE, MSE и R2 на обущающем и тестовом наборе

5.3 Вывести model.coef_ и model.intercept_

5.4 Построить график (barh) с важностью коэфициентов при соответствующих признаках. Желательно, чтобы сверху вниз шли коэффициенты по убыванию, а названия соответствующих признаков подписаны по вертикальной оси.

## Классификация

Ссылки на теорию:

- https://habr.com/ru/post/685014/
- https://habr.com/ru/company/ods/blog/328372/
- https://habr.com/ru/post/491326/

0.  Загрузите наборы данных, подходящие для многоклассовой и бинарной классификации. Опишите данные, что обозначает каждый столбец, какого типа (категориальный, вещественный, целочисленный, бинарный) данный столбец, при необходимости для категориальных в том числе бинарных признаков привести расшифровку их значений. В общем, проведите "визуальный" анализ ваших данных.

**1. Многоклассовая классификация:**

1.1 На любом датасете продемонстрировать многоклассовую классификацию с помощью ЛЮБОГО алгоритма. Оценить accuracy.

1.2. То же самое с использованием кросс-валидации

1.3. Поиграться с гиперпараметром, связанным со сложностью модели (в случае kNN — k_neighbours) и построить график accuracy для train set и test_set для разных значений гиперпараметра (в лекции есть пример построения подобного графика для kNN и датасета cancer, но можно строить как угодно такой график)

1.4. Построить confusion matrix (это вроде матрицы TN / TP/ FN/ FP, только расширенной для многоклассовой классификации, чтобы понять в какие стороны чаще ошибается классификатор). ([пример](https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html)):

**2. Бинарная классификация.**

На любом (но уже другом, бинарном) датасете продемонстрировать бинарную классификацию (можно использовать любой алгоритм), посчитать метрики: accuracy, precision, recall, f-measure, а также составить `classification_report`



**3. Использование нормализации признаков**
В обоих задачах (бинарной и многоклассовой классификации) попробовать также вариант с предварительной нормализацией признаков (`StandardScaler`или `MinMaxScaler`). Оценить метрики качества в случае с нормализованными признаками и ненормализованными.